In [1]:
import numpy as np

from ppfft.tools.new_fft import new_fft, new_fft2


# How do we pad?

We have a sequence $x \in \mathbb{R}^n$. Let $m \in \mathbb{N}, m \geq n$. We wish to compute:
$$
\forall k \in \mathcal{D}(m), \sum_{u \in \mathcal{D}(n)} x(u) \exp\left(-2i \pi \dfrac{u k}{m} \right)
$$

This can be obtained by zero padding $x$ on both sides, before applying the new FFT. In this notebook we implement the correct way of padding.

## In 1D

In [2]:
def naive_fourier_pad(x, m):
    n = len(x)

    if n % 2 == 0:
        half_n = n//2
        # from -(n//2) to n//2 - 1 : 2*(n//2) = n points
        u = np.arange(-half_n, half_n)
    else:
        half_n = n//2
        # from -(n//2) to n//2 : 2*(n//2) + 1 = n points
        u = np.arange(-half_n, half_n + 1)

    if m % 2 == 0:
        half_m = m//2
        k = np.arange(-half_m, half_m)
    else:
        half_m = m//2
        k = np.arange(-half_m, half_m + 1)

    ku = np.exp(-2j * np.pi * np.einsum("k,u->ku", k, u) / m)

    res = np.einsum("u,ku->k", x, ku)

    return res


In [3]:
def clever_fourier_pad(x, m):
    """
    Zero pads x from the shape (n,) to the shape (m,).
    """
    n = len(x)
    q_n, r_n = divmod(n, 2)
    q_m, r_m = divmod(m, 2)

    if r_n == r_m:
        padded_x = np.pad(x, (q_m - q_n, q_m - q_n))
        return new_fft(padded_x)
    else:
        padded_x = np.pad(x, (q_m - q_n, q_m - q_n + r_m - r_n))
        return new_fft(padded_x)


In [4]:
x = np.random.rand(10)
m = 15
print(naive_fourier_pad(x, m) - clever_fourier_pad(x, m))


[ 9.99200722e-16-5.96744876e-16j  5.55111512e-17+1.77635684e-15j
  6.66133815e-16-3.33066907e-16j  5.82867088e-16-3.33066907e-16j
 -4.44089210e-16+1.22124533e-15j  0.00000000e+00-4.44089210e-16j
  6.66133815e-16+2.22044605e-16j -8.88178420e-16+0.00000000e+00j
  4.44089210e-16+0.00000000e+00j  0.00000000e+00+4.44089210e-16j
 -4.44089210e-16-1.22124533e-15j  6.38378239e-16+3.33066907e-16j
  4.99600361e-16+5.55111512e-16j  5.55111512e-17-1.77635684e-15j
  1.11022302e-15+4.85722573e-16j]


In [5]:
x = np.random.rand(10)
m = 16
print(naive_fourier_pad(x, m) - clever_fourier_pad(x, m))


[ 1.11022302e-16-9.16386139e-17j -5.55111512e-17+3.88578059e-16j
 -6.66133815e-16+6.66133815e-16j  3.88578059e-16+0.00000000e+00j
  0.00000000e+00+8.32667268e-17j -5.55111512e-16+6.66133815e-16j
 -2.22044605e-16-2.22044605e-16j  2.22044605e-16+2.77555756e-16j
 -8.88178420e-16+0.00000000e+00j  2.22044605e-16-2.77555756e-16j
 -2.22044605e-16+2.22044605e-16j -5.55111512e-16-6.66133815e-16j
  0.00000000e+00-8.32667268e-17j  3.88578059e-16+0.00000000e+00j
 -6.66133815e-16-6.66133815e-16j -5.55111512e-17-3.88578059e-16j]


In [6]:
x = np.random.rand(11)
m = 15
print(naive_fourier_pad(x, m) - clever_fourier_pad(x, m))


[ 2.33146835e-15+0.00000000e+00j -4.44089210e-16-9.99200722e-16j
  8.88178420e-16+2.22044605e-16j  1.33226763e-15+7.77156117e-16j
 -4.44089210e-16-2.22044605e-16j -8.88178420e-16-1.38777878e-17j
  4.44089210e-16+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  4.44089210e-16+0.00000000e+00j -4.44089210e-16-1.38777878e-16j
 -4.44089210e-16+2.22044605e-16j  1.55431223e-15-5.68989300e-16j
  6.66133815e-16-2.22044605e-16j -4.44089210e-16+9.99200722e-16j
  2.10942375e-15-2.22044605e-16j]


In [7]:
x = np.random.rand(11)
m = 16
print(naive_fourier_pad(x, m) - clever_fourier_pad(x, m))


[-2.22044605e-16+4.26652017e-16j -1.11022302e-16+0.00000000e+00j
 -1.11022302e-16-4.99600361e-16j  0.00000000e+00-5.55111512e-17j
 -1.11022302e-16+2.22044605e-16j -3.33066907e-16-1.94289029e-16j
  5.55111512e-17+1.11022302e-16j  4.44089210e-16+1.11022302e-16j
 -8.88178420e-16+0.00000000e+00j  4.44089210e-16-1.11022302e-16j
  5.55111512e-17-1.11022302e-16j -3.33066907e-16+1.94289029e-16j
 -1.11022302e-16-2.22044605e-16j  0.00000000e+00+5.55111512e-17j
 -1.11022302e-16+4.99600361e-16j -1.11022302e-16+0.00000000e+00j]


# In 2D

In [8]:
def naive_fourier_2D_pad(a, new_shape):
    n, m = a.shape
    new_n, new_m = new_shape

    if n % 2 == 0:
        half_n = n//2
        # from -(n//2) to n//2 - 1 : 2*(n//2) = n points
        u = np.arange(-half_n, half_n)
    else:
        half_n = n//2
        # from -(n//2) to n//2 : 2*(n//2) + 1 = n points
        u = np.arange(-half_n, half_n + 1)

    if new_n % 2 == 0:
        half_new_n = new_n//2
        k = np.arange(-half_new_n, half_new_n)
    else:
        half_new_n = new_n//2
        k = np.arange(-half_new_n, half_new_n + 1)

    ku = np.exp(-2j * np.pi * np.einsum("k,u->ku", k, u) / new_n)

    if m % 2 == 0:
        half_m = m//2
        # from -(n//2) to n//2 - 1 : 2*(n//2) = n points
        v = np.arange(-half_m, half_m)
    else:
        half_m = m//2
        # from -(n//2) to n//2 : 2*(n//2) + 1 = n points
        v = np.arange(-half_m, half_m + 1)

    if new_m % 2 == 0:
        half_new_m = new_m//2
        l = np.arange(-half_new_m, half_new_m)
    else:
        half_new_m = new_m//2
        l = np.arange(-half_new_m, half_new_m + 1)

    lv = np.exp(-2j * np.pi * np.einsum("l,v->lv", l, v) / new_m)

    res = np.einsum("uv,ku,lv->kl", a, ku, lv)

    return res


In [9]:
def clever_fourier_2D_pad(a, new_shape):
    n, m = a.shape
    new_n, new_m = new_shape

    q_n, r_n = divmod(n, 2)
    q_new_n, r_new_n = divmod(new_n, 2)

    if r_n == r_new_n:
        padded_a = np.pad(a, ((q_new_n - q_n, q_new_n - q_n), (0, 0)))
    else:
        padded_a = np.pad(
            a, ((q_new_n - q_n, q_new_n - q_n + r_new_n - r_n), (0, 0)))

    q_m, r_m = divmod(m, 2)
    q_new_m, r_new_m = divmod(new_m, 2)

    if r_m == r_new_m:
        padded_a = np.pad(padded_a, ((0, 0), (q_new_m - q_m, q_new_m - q_m)))
    else:
        padded_a = np.pad(
            padded_a, ((0, 0), (q_new_m - q_m, q_new_m - q_m + r_new_m - r_m)))

    return new_fft2(padded_a)


In [10]:
a_shapes = [(4, 4), (5, 4), (4, 5), (5, 5)]
new_shapes = [(6, 6), (7, 6), (6, 7), (7, 7)]

for a_shape in a_shapes:
    for new_shape in new_shapes:
        a = np.random.random(a_shape)
        print(np.max(np.abs(naive_fourier_2D_pad(a, new_shape) -
                            clever_fourier_2D_pad(a, new_shape))))


1.4229449672489473e-15
1.7763568394002505e-15
2.220446049250313e-15
3.552713678800501e-15
1.7790111516265277e-15
2.220446049250313e-15
1.7763568394002505e-15
1.7763568394002505e-15
1.7798229048217483e-15
1.7763568394002505e-15
1.006115688561674e-15
1.790180836524724e-15
2.220446049250313e-15
1.9389211565826713e-15
1.7798229048217483e-15
1.807312143953211e-15


# General padding function

In [11]:
def pad(x: np.ndarray, new_shape: tuple) -> np.ndarray:
    """
    Zero-pads the array x in a way that is compatible with ``new_fft`` and ``new_fft2``.
    """

    res = np.copy(x)
    n_dim = res.ndim

    assert n_dim == len(new_shape)

    pad_width = [(0, 0)] * n_dim

    for i, (n, new_n) in enumerate(zip(x.shape, new_shape)):

        q_n, r_n = divmod(n, 2)
        q_new_n, r_new_n = divmod(new_n, 2)

        if r_n == r_new_n:
            pad_i = (q_new_n - q_n, q_new_n - q_n)
        else:
            pad_i = (q_new_n - q_n, q_new_n - q_n + r_new_n - r_n)

        pad_width[i] = pad_i

    res = np.pad(res, pad_width)

    return res


In [12]:
def new_fourier_2D_pad(a, new_shape):
    return new_fft2(pad(a, new_shape))


In [13]:
a_shapes = [(4, 4), (5, 4), (4, 5), (5, 5)]
new_shapes = [(6, 6), (7, 6), (6, 7), (7, 7)]

for a_shape in a_shapes:
    for new_shape in new_shapes:
        a = np.random.random(a_shape)
        print(np.max(np.abs(new_fourier_2D_pad(a, new_shape) -
                            clever_fourier_2D_pad(a, new_shape))))


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


## Adjoint of ``pad``

In [14]:
def matrix_dot(a, b):
    return np.sum(np.conjugate(a) * b)


In [15]:
def adj_pad(x, original_shape):

    res = np.copy(x)
    n_dim = res.ndim

    assert n_dim == len(original_shape)

    for i, (n, original_n) in enumerate(zip(x.shape, original_shape)):

        q_n, r_n = divmod(n, 2)
        q_original_n, r_original_n = divmod(original_n, 2)

        if r_n == r_original_n:
            # The padding was: (q_n - q_original_n, q_n - q_original_n)
            indices = np.arange(q_n - q_original_n, n - (q_n - q_original_n))
        else:
            # The padding was: (q_n - q_original_n, q_n - q_original_n + r_n - r_original_n)
            indices = np.arange(q_n - q_original_n, n -
                                (q_n - q_original_n + r_n - r_original_n))

        res = np.take(res, indices, axis=i)

    return res


In [16]:
shapes = [(4,), (5,)]
new_shapes = [(6,), (7,)]

for shape in shapes:
    for new_shape in new_shapes:
        x = np.random.random(shape)
        y = np.random.random(new_shape)
        print(np.vdot(pad(x, new_shape), y) -
              np.vdot(x, adj_pad(y, shape)))


0.0
0.0
0.0
0.0


In [17]:
shapes = [(4, 4), (4, 5), (5, 4), (5, 5)]
new_shapes = [(6, 6), (6, 7), (7, 6), (7, 7)]
err_list = []

for shape in shapes:
    for new_shape in new_shapes:
        x = np.random.random(shape)
        y = np.random.random(new_shape)
        err_list.append(matrix_dot(pad(x, new_shape), y) -
                        matrix_dot(x, adj_pad(y, shape)))

np.max(np.abs(err_list))


1.7763568394002505e-15